In [1]:
import random
import copy
from collections import namedtuple
from dataclasses import dataclass
import datetime
import typing
import functools
from pprint import pprint

In [2]:
import jax
import jax.numpy as jnp
from jax import grad, value_and_grad, jit, vmap
from jax.experimental import optimizers
from jax.experimental import stax
import optax
import haiku as hk
from jax.tree_util import tree_flatten

import pyspiel
import open_spiel
import dm_env
import acme
import acme.wrappers
import acme.jax.utils
from acme.agents import agent as acme_agent
from acme.agents import replay as acme_replay
from acme.environment_loops.open_spiel_environment_loop import OpenSpielEnvironmentLoop
from acme.wrappers.open_spiel_wrapper import OpenSpielWrapper

from tqdm.notebook import tqdm
import numpy as np
import trueskill

In [3]:
import moozi as mz

In [4]:
# %run hardware_sanity_check.ipynb

In [5]:
# OpenSpiel environment, not using it for now since not supported by the latest relased Acme
# raw_env = open_spiel.python.rl_environment.Environment('catch(columns=8,rows=4)')
raw_env = open_spiel.python.rl_environment.Environment('cliff_walking')
env = acme.wrappers.open_spiel_wrapper.OpenSpielWrapper(raw_env)
env = acme.wrappers.SinglePrecisionWrapper(env)
env_spec = acme.specs.make_environment_spec(env)
dim_action = env_spec.actions.num_values
dim_image = env_spec.observations.observation.shape
dim_repr = 3

In [6]:
nn_spec = mz.nn.NeuralNetworkSpec(
    dim_image=dim_image,
    dim_repr=dim_repr,
    dim_action=dim_action
)
print(nn_spec)
network = mz.nn.get_network(nn_spec)
optimizer = optax.adam(1e-4)

NeuralNetworkSpec(dim_image=(400,), dim_repr=3, dim_action=4)


In [7]:
batch_size = 128
n_steps=5
# reverb_replay = acme_replay.make_reverb_prioritized_nstep_replay(
#     env_spec, batch_size=batch_size, n_step=5)
reverb_replay = acme_replay.make_reverb_prioritized_sequence_replay(
    env_spec, batch_size=batch_size)
actor = mz.actor.RandomActor(reverb_replay.adder)
learner = mz.learner.MooZiLearner(
    network=network,
    loss_fn=mz.loss.initial_inference_value_loss,
    optimizer=optimizer,
    data_iterator=reverb_replay.data_iterator,
    random_key=jax.random.PRNGKey(996),
)

In [8]:
agent = acme_agent.Agent(
    actor=actor, learner=learner, min_observations=100, observations_per_step=1)

In [9]:
loop = OpenSpielEnvironmentLoop(environment=env, actors=[agent])

In [10]:
loop.run_episode()

{'episode_length': 1,
 'episode_return': array([-100.], dtype=float32),
 'steps_per_second': 8.459891041919212,
 'episodes': 1,
 'steps': 1}

In [11]:
loop.run(num_episodes=1000)

ValueError: Incompatible shapes for broadcasting: ((128, 121, 1), (1, 128, 121))